In [ ]:
#PROJETO TRADING PYTHON

In [24]:
#Bibliotecas
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
from datetime import date

pd.options.mode.chained_assignment = None #Retirar alguns avisos

data_atual = date.today() # Mostra Data Atual


2024-01-21


In [ ]:
#Ativo escolhido
try:
    ativo = input("Digite um ativo")
    #Alterar buscando dentro o yahoo finance(código de negociação). Exemplo.: 'PETR4.SA'  
    dataInicio = input("Digite uma data de origem do ativo. Ex.:(2000-12-31) ---> Se não quiser adicionar data digite (nada)")
       
except(RuntimeError):
    pass

In [25]:
dataFim = data_atual
if(dataInicio == "nada"):
    dataInicio = "2000-12-31" #Pegar a partir da criação
    dados_ativo = yf.download(ativo,dataInicio,dataFim)
else:
    dados_ativo = yf.download(ativo, dataInicio, dataFim) #ativo(var),DataInicio,DataFim
dados_ativo

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-11-09,308.644989,329.451996,307.056000,320.884003,320.884003,893249984
2017-11-10,320.670990,324.717987,294.541992,299.252991,299.252991,885985984
2017-11-11,298.585999,319.453003,298.191986,314.681000,314.681000,842300992
2017-11-12,314.690002,319.153015,298.513000,307.907990,307.907990,1613479936
2017-11-13,307.024994,328.415009,307.024994,316.716003,316.716003,1041889984
...,...,...,...,...,...,...
2024-01-16,2510.627197,2613.566895,2500.003906,2587.691162,2587.691162,11063317095
2024-01-17,2587.044678,2592.737061,2508.432861,2528.369385,2528.369385,10441017520
2024-01-18,2528.593262,2546.263916,2426.135498,2467.018799,2467.018799,11900028080


In [ ]:
dados_ativo['Adj Close'].plot() #Capturar dados fechamento ajustado(adj Close)

In [ ]:
#Calcular Retornos
dados_ativo['retornos'] = dados_ativo['Adj Close'].pct_change().dropna() #Criar uma nova coluna que pega a coluna 'Adj Close' = retorno diário do Ativo
dados_ativo

In [ ]:
#TESTE
retorno = -1
filtrando_retorno = lambda x: x if x> 0 else 0 #lambda= retorna a ele mesmo, função auto-referente, x = valor de exemplo

filtrando_retorno(retorno)

In [ ]:
#Separar retornos Positivos e Negativos
dados_ativo['retornos_positivos'] = dados_ativo['retornos'].apply(lambda x: x if x > 0 else 0) #Aplicando a funcao lambda para valores positivos
dados_ativo['retornos_negativos'] = dados_ativo['retornos'].apply(lambda x: abs(x) if x < 0 else 0) #abs(x) = pega o valor absoluto

dados_ativo

In [ ]:
#Calcular média dos retornos positivos e negativos nos últimos 22 dias
dados_ativo['media_retornos_positivos'] = dados_ativo['retornos_positivos'].rolling(window = 22).mean()#cria um coluna com uma janela de 22 dias e calcula a média
dados_ativo['media_retornos_negativos'] = dados_ativo['retornos_negativos'].rolling(window = 22).mean()

dados_ativo = dados_ativo.dropna() #Mostrar sem o NAN

dados_ativo

In [ ]:
#Calcular RSI
#Fórmula = 100 - 100(1 + mediaRetornosPositivos/mediaRetornosNegativos)

dados_ativo['RSI'] = (100 - 100/(1+ dados_ativo['media_retornos_positivos']/dados_ativo['media_retornos_negativos']))

dados_ativo.head(50) #Printa 50 dados

In [ ]:
#Sinal de compra e venda (abaixo de 30 = sinal de compra)
dados_ativo.loc[dados_ativo['RSI'] < 30, 'compra'] = 'sim' #Verifica se o valor de RSI é maior ou menor que 30, sinalizando ou não a compra
dados_ativo.loc[dados_ativo['RSI'] > 30, 'compra'] = 'nao' #.loc = localizar um valor

dados_ativo

In [ ]:
#Buscar datas de compra e de vendas
datas_compra = []
datas_venda = []

for i in range(len(dados_ativo)):
  print(i)

  if "sim" in dados_ativo['compra'].iloc[i]: #se o sim está dentro dessa coluna, e então ele pegará os dados de abertura do próximo dia
    datas_compra.append(dados_ativo.iloc[i+1].name) #.name = valor da coluna

datas_compra

In [ ]:
#Buscar datas de venda:
#     Se valor de RSI está acima de 40
#     10 dias de operação

data_compra = []
data_venda = []

for i in range(len(dados_ativo)):

    if "sim" in dados_ativo['compra'].iloc[i]:

        data_compra.append(dados_ativo.iloc[i+1].name) # +1 porque a gente compra no preço de abertura do dia seguinte.

        for j in range(1, 11):

            if dados_ativo['RSI'].iloc[i + j] > 40: #vendo se nos proximos 10 dias o RSI passa de 40

                data_venda.append(dados_ativo.iloc[i + j + 1].name) #vende no dia seguinte q bater 40
                break

            elif j == 10:
                data_venda.append(dados_ativo.iloc[i + j + 1].name)



data_venda

In [ ]:
#Visualização de pontos de compra
plt.figure(figsize = (12, 5))
plt.scatter(dados_ativo.loc[data_compra].index, dados_ativo.loc[data_compra]['Adj Close'], marker = "^",c='g')
plt.plot(dados_ativo['Adj Close'], alpha = 0.7)

In [ ]:
lucros = dados_ativo.loc[data_venda]['Open'].values/dados_ativo.loc[data_compra]['Open'].values - 1

lucros

In [ ]:
operacoes_vencedoras = len(lucros[lucros > 0])/len(lucros)

operacoes_vencedoras #Neste caso 58% dos casos a gente ganha aplicando tal modelo

In [ ]:
media_ganhos = np.mean(lucros[lucros > 0])

media_ganhos * 100 #7.02% média de ganho

In [ ]:
media_perdas = abs(np.mean(lucros[lucros < 0]))

media_perdas * 100 #5.93% média de perda

In [ ]:
expectativa_matematica_modelo = (operacoes_vencedoras * media_ganhos) - ((1 - operacoes_vencedoras) * media_perdas)

expectativa_matematica_modelo * 100 #Para cada operacao, quanto você irá ganhar (em relação a média de ganhos e a média de perdas), nesse exemplo.: 1.48%

In [ ]:
performance_acumulada = (np.cumprod((1 + lucros)) - 1)

performance_acumulada * 100 #acumulando os lucros (rentabilidade acumulada)

In [ ]:
plt.figure(figsize = (12, 5))
plt.plot(data_compra, performance_acumulada) #Visualização do lucro acumulado 12.: 1200% de valorização

In [ ]:
retorno_buy_and_hold = dados_ativo['Adj Close'].iloc[-1]/dados_ativo['Adj Close'].iloc[0] - 1

retorno_buy_and_hold * 100 #Quanto rendeu desde da data de inicio